## Prepare Dataset List

In [1]:
import os
import random

import torch
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset as dataset

import SimpleITK as sitk
import scipy.ndimage as ndimage

In [25]:
size = 30
lower = 0

In [132]:
class Dataset(dataset):
    def  __init__ (self , ct_dir, seg_dir):
        self.ct_list = os.listdir(ct_dir)
        self.seg_list = list(map(lambda x: x.replace('img', 'label'), self.ct_list))

        self.ct_list = list(map(lambda x: os.path.normpath(os.path.join(ct_dir, x)), self.ct_list))
        self.seg_list = list(map(lambda x: os.path.normpath(os.path.join(seg_dir, x)), self.seg_list))

    def __getitem__(self, index):
        """
        :param index:
        :return: torch.Size([B, 1, 48, 256, 256]) torch.Size([B, 48, 256, 256])
        """
        
        ct_path = self.ct_list[index]
        seg_path  = self.seg_list[index]

        # Read CT and gold standard into memory
        ct  =  sitk.ReadImage(ct_path , sitk.sitkInt16)
        seg  =  sitk.ReadImage(seg_path , sitk.sitkUInt8)

        ct_array = sitk.GetArrayFromImage(ct)
        seg_array = sitk.GetArrayFromImage(seg)

        #Randomly select 48 slices in the slice plane
        start_slice = random.randint(0, ct_array.shape[0] - size)
        end_slice = start_slice + size - 1

        ct_array = ct_array[start_slice:end_slice + 1, :, :]
        seg_array = seg_array [start_slice:end_slice + 1 , :, :]

#         # Randomly rotate within 5 degrees with probability 0.5
#         # If the angle is negative, it will rotate clockwise, if the angle is positive, it will rotate counterclockwise
        if random.uniform(0, 1) >= 0.5:
            angle = random.uniform(-5, 5)
            ct_array = ndimage.rotate(ct_array, angle, axes =(1, 2), reshape = False , cval = lower);
            seg_array = ndimage.rotate(seg_array, angle, axes = (1, 2), reshape = False, cval = 0);

        #There is a probability of 0.5 without any modification, and the remaining 0.5 randomly selects a patch with a size of 0.8-0.5 and enlarges it to 256*256
#         if random.uniform(0, 1) >= 0.5:
#             ct_array, seg_array = self.zoom(ct_array, seg_array, patch_size=random.uniform(0.5, 0.8))
        
#         ct_array = F.interpolate(ct_array, (size, 256, 256), mode='trilinear').squeeze().detach().numpy()
#         seg_array = F.interpolate(seg_array, (size, 256, 256), mode='trilinear').squeeze().detach().numpy()

#         with torch.no_grad():
#             ct_array = torch.FloatTensor(ct_array).unsqueeze(dim=0).unsqueeze(dim=0)
#             ct_array = Variable(ct_array)
#             ct_array = F.interpolate(ct_array, (size, 256, 256), mode='trilinear').squeeze().detach().numpy()

#             seg_array = torch.FloatTensor(seg_array).unsqueeze(dim=0).unsqueeze(dim=0)
#             seg_array = Variable(seg_array)
#             seg_array = F.interpolate(seg_array, (size, 256, 256), mode='trilinear').squeeze().detach().numpy()
        

        # After processing, convert array to tensor
        ct_array = torch.FloatTensor(ct_array).unsqueeze(0)
        seg_array = torch.FloatTensor(seg_array)
        
        ct_array = F.interpolate(ct_array, (size, 256, 256), mode='trilinear').squeeze().detach().numpy()
        seg_array = F.interpolate(seg_array, (size, 256, 256), mode='trilinear').squeeze().detach().numpy()
        
        # ct_array = torch.FloatTensor(ct_array)
        # seg_array = torch.FloatTensor(seg_array)
        
        print("{} {}".format(ct_array.shape, seg_array.shape))

        return ct_array, seg_array

    def __len__(self):
        return len(self.ct_list)

    def zoom(self, ct_array, seg_array, patch_size):

        length = int(256*patch_size)

        x1 = int(random.uniform(0, 255-length))
        y1 = int(random.uniform(0, 255-length))

        x2 = x1 + length
        y2 = y1 + length

        ct_array = ct_array[:, x1:x2 + 1, y1:y2 + 1]
        seg_array = seg_array[:, x1:x2 + 1 , y1:y2 + 1]    

        with torch.no_grad():

            ct_array = torch.FloatTensor(ct_array).unsqueeze(dim=0).unsqueeze(dim=0)
            ct_array = Variable(ct_array)
            ct_array = F.interpolate(ct_array, (size, 256, 256), mode='trilinear').squeeze().detach().numpy()

            seg_array = torch.FloatTensor(seg_array).unsqueeze(dim=0).unsqueeze(dim=0)
            seg_array = Variable(seg_array)
            seg_array = F.interpolate(seg_array, (size, 256, 256), mode='trilinear').squeeze().detach().numpy()

            return ct_array, seg_array

In [133]:
ct_dir = "E:/skripsi/Dataset/train/CT"
seg_dir = "E:/skripsi/Dataset/train/GT"

train_ds = Dataset(ct_dir, seg_dir)

In [134]:
train_ds.seg_list

['E:\\skripsi\\Dataset\\train\\GT\\label-0.nii',
 'E:\\skripsi\\Dataset\\train\\GT\\label-1.nii',
 'E:\\skripsi\\Dataset\\train\\GT\\label-10.nii',
 'E:\\skripsi\\Dataset\\train\\GT\\label-12.nii',
 'E:\\skripsi\\Dataset\\train\\GT\\label-13.nii',
 'E:\\skripsi\\Dataset\\train\\GT\\label-14.nii',
 'E:\\skripsi\\Dataset\\train\\GT\\label-15.nii',
 'E:\\skripsi\\Dataset\\train\\GT\\label-17.nii',
 'E:\\skripsi\\Dataset\\train\\GT\\label-18.nii',
 'E:\\skripsi\\Dataset\\train\\GT\\label-19.nii',
 'E:\\skripsi\\Dataset\\train\\GT\\label-20.nii',
 'E:\\skripsi\\Dataset\\train\\GT\\label-21.nii',
 'E:\\skripsi\\Dataset\\train\\GT\\label-22.nii',
 'E:\\skripsi\\Dataset\\train\\GT\\label-23.nii',
 'E:\\skripsi\\Dataset\\train\\GT\\label-25.nii',
 'E:\\skripsi\\Dataset\\train\\GT\\label-26.nii',
 'E:\\skripsi\\Dataset\\train\\GT\\label-27.nii',
 'E:\\skripsi\\Dataset\\train\\GT\\label-28.nii',
 'E:\\skripsi\\Dataset\\train\\GT\\label-3.nii',
 'E:\\skripsi\\Dataset\\train\\GT\\label-30.nii',
 'E

## Define Model

In [135]:
import torch.nn as nn

In [136]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class AttentionBlock(nn.Module):
    def __init__(self, in_channels, skip_channels, out_channels):
        super(AttentionBlock, self).__init__()
        self.W_g = nn.Sequential(
            nn.Conv2d(skip_channels, out_channels, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(out_channels)
        )

        self.W_x = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(out_channels)
        )

        self.psi = nn.Sequential(
            nn.Conv2d(out_channels, 1, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x, skip):
        g = self.W_g(skip)
        x = self.W_x(x)

        psi = self.relu(g + x)
        psi = self.psi(psi)

        return x * psi

class UpConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UpConv, self).__init__()
        self.up_conv = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)

    def forward(self, x):
        return self.up_conv(x)

class AttU_Net(nn.Module):
    def __init__(self, in_channels=2, out_channels=2, init_features=32):
        super(AttU_Net, self).__init__()

        self.down1 = DoubleConv(in_channels, init_features)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.down2 = DoubleConv(init_features, init_features * 2)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.down3 = DoubleConv(init_features * 2, init_features * 4)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.down4 = DoubleConv(init_features * 4, init_features * 8)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.center = DoubleConv(init_features * 8, init_features * 16)

        self.up4 = UpConv(init_features * 16, init_features * 8)
        self.att4 = AttentionBlock(init_features * 8, init_features * 4, init_features * 8)
        self.up3 = UpConv(init_features * 8, init_features * 4)
        self.att3 = AttentionBlock(init_features * 4, init_features * 2, init_features * 4)
        self.up2 = UpConv(init_features * 4, init_features * 2)
        self.att2 = AttentionBlock(init_features * 2, init_features * 2, init_features * 2)
        self.up1 = UpConv(init_features * 2, init_features)
        self.att1 = AttentionBlock(init_features, init_features, init_features)

        self.final_conv = nn.Conv2d(init_features, out_channels, kernel_size=1)
        
    def forward(self, x):
        skip1 = self.down1(x)
        pool1 = self.pool1(skip1)

        skip2 = self.down2(pool1)
        pool2 = self.pool2(skip2)

        skip3 = self.down3(pool2)
        pool3 = self.pool3(skip3)

        skip4 = self.down4(pool3)
        pool4 = self.pool4(skip4)

        center = self.center(pool4)

        up4 = self.up4(center)
        att4 = self.att4(up4, skip4)

        up3 = self.up3(att4)
        att3 = self.att3(up3, skip3)

        up2 = self.up2(att3)
        att2 = self.att2(up2, skip2)

        up1 = self.up1(att2)
        att1 = self.att1(up1, skip1)

        out = self.final_conv(att1)

        return out

## Training

In [137]:
import torch.optim as optim
from torch.utils.data import DataLoader

In [138]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [139]:
device

device(type='cuda')

In [140]:
# Define hyperparameters
num_epochs = 50
batch_size = 4
learning_rate = 1e-4

# Define loss function and device
loss_fn = nn.CrossEntropyLoss() # use cross-entropy loss for multi-class segmentation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define dataset and dataloader
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=False)

# Define model and optimizer
model = AttU_Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    total_loss = 0.0
    for i, batch in enumerate(train_loader):
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss:.4f}")

ValueError: Input and output must have the same number of spatial dimensions, but got input with with spatial dimensions of [160, 160] and output size of (30, 256, 256). Please provide input tensor in (N, C, d1, d2, ...,dK) format and output size in (o1, o2, ...,oK) format.